The general principle we are going to apply in this project is refering to our variables as lists of matrices.

For example, for $x$, the variable corresponds to a list of $p$ $n \times n$ matrices.
Thus, the entry $x_{kij}$ represents wether office $i$ moves to office $j$ in phase k.

We then vectorize all these matrices in one big $p \times n \times n$ vector.

In [ ]:
import cvxpy as cvx # A good library with open-source solvers and modeling language
import numpy as np # Basic library for array structures
from numpy.random import multivariate_normal, randn, uniform, choice # Probability distributions
from scipy.linalg import norm # Efficient norm calculation
from scipy.linalg import solve # Linear system solve
from math import sqrt
import sys
import time # Measure elapsed time

## 5 - office version

### Basic things

In [ ]:
p = 6
w = 5
n = 5
c = 5

N = p * n**2

offices_per_wing = np.cumsum(np.ones(w)) - 1

colors=["Unaffected","MIDO", "LAMSADE", "the student association", "the presidency"]

print(offices_per_wing)

[0. 1. 2. 3. 4.]


Helper functions

In [ ]:
def get_row(phase, rownb):
  p1 = np.zeros(phase * n**2)
  p2 = np.ravel(np.array([[1 if j == rownb else 0 for j in range(n)] for _ in range(n)]).T)
  p3 = np.zeros((p - phase - 1) * n**2)

  return np.block([p1, p2, p3])

def get_column(phase, colnb):
  p1 = np.zeros(phase * n**2)
  p2 = np.ravel(np.array([[1 if j == colnb else 0 for j in range(n)] for _ in range(n)]))
  p3 = np.zeros((p - phase - 1) * n**2)

  return np.block([p1, p2, p3])

### Constraint 0 : Initial positions

In [ ]:
# init setting constraints - 1 entry
cstrInit = np.empty((n, n))
cstrInit[0] = np.zeros(n)
for i in range(1, n):
  vec = [1 if j == i else 0 for j in range(n)]
  cstrInit[i] = vec
cstrInit = np.ravel(cstrInit)


b = np.array([4])
A = np.block([cstrInit, np.zeros((p-1)* n**2)])

Defining the initial assignment matrix, we will use it in phase 0.

1 constraint.

### Constraint 1 : don't move to a wing under construction

In [ ]:
# office j empty - p constr
constraint1 = np.empty((p - 1,N))
for i in range(p - 1):
  constraint1[i] = get_column(i, i)

b = np.block([
    b,
    np.zeros(p - 1)])

A = np.block([[A],
 [constraint1]])

When a wing is under construction, no office should move to any office in that wing.

This corresponds to having column $i$ being full of zeros if office $i$ is under construction. Since all entries of $x$ are non-negative, we can simply say that the sum of the entries in the column should be 0.

$n$ constraints.

### Constraint 2 : don't move to the same office as somebody else

In [ ]:
allcols = np.empty((n*(p-2), N))
for phase in range(1, p-1):
  for office in range(n):
    allcols[(phase-1)*n + office] = get_column(phase, office)

# We will later upperbound this column selection

Here we simply select all columns, which we are going to upper-bound by one (later in the constraints) to make sure that we don't have several offices moving to the same destination in the same phase.

$n \times (p-2)$ constraints.

### Constraint 3 : phase to phase correspondance

In [ ]:
# If someone was here, move (even if on yourself)
# Otherwise nothing should come out of an empty office
constraint3 = np.empty(((p - 1) * n, N))
for i in range(1, p) :
  for t in range(n):
    cst = - get_column(i-1, t)
    # A negative column in the previous phase

    # If the office is located in the wing under construction, move it somewhere else
    if (t > offices_per_wing[i - 1] and t <= offices_per_wing[i]) or i == p-1:
      cst += get_row(i,t)

    else : # Otherwise move it to the same place as before
      cst[i*n**2 + t*n + t] = 1

    constraint3[(i - 1) * n + t] = cst

A = np.block([[A],
              [constraint3]])
b = np.block([b, np.zeros((p-1) * n)])

The idea behind this constraint is that nothing should come out of an empty office and that something should come out of an occupied office.

However, if the office in question is not under construction, then we don't need to move anywhere else, thus we force the office to "move" exactly where it already is. This helps us keeping track of where the offices are even if they have not changed locations recently.

$(p-1) \times n$ constraints.

###Constraint 4 : final positions

In [ ]:
## COLUMN CONSTRAINTS

#Column 1 in the last phase  == 0
c1Final = get_column(p - 1, 0)


#Columns 2 to 5 in the last phase == 1
cOtherColsFinal = np.empty((n - 1, N))
for i in range(1, n):
  cst = get_column(p-1, i)
  #print(cst.reshape(6, 5, 5))
  cOtherColsFinal[i - 1] = cst


b = np.block([b,
              0,
              np.ones(n - 1)])
A = np.block([ [A],
              [c1Final],
              [cOtherColsFinal]
               ])

$n$ constraints.

### Color constraints

For color representation, we are going to need to additional variables :


*   $y$ is a $p \times n \times c$ matrix, with $y_{k,i,j}$ representing whether office $i$ is of color $j$ in phase $k$.
*   $w$ is a $p \times n \times c \times n$ matrix which represents the products of certain entries of $x$ and $y$. This variable is going to be of use in the enforcing phase to phase correspondance for the colors.

As said in the beginning, we are going to reference this variables as matrices to help clarify what entries we are talking about. But in practice, we vectorize them for the solving part.


The phase to phase correspondance contraint for $y$ is the following :
$$y_{k,i,j} = ∑_{e=1}^n w_{k,i,j,e}$$

$NB$ : this formula only holds for $y$ entries with $ k \ge 1$ since there is no correspondance constraint with any prebvious phase for $y$ entries in phase 0.

Y constraints

In [ ]:
# Initial and final color assignment constraints
InitAy = np.block([np.eye(n*c), np.zeros((n*c, (p-1)*n*c))]) # Selects y entries in the first phase
FinalAy = np.block([np.zeros((n*c, (p-1)*n*c)), np.eye(n*c)]) # Selects y entries in the last phase

init_colors = cstrInit
final_colors = init_colors

$2 \times n \times c$ constraints.

In [ ]:

# For each entry of y, select the corresponding n entries of w
Aw = np.empty((n * c * (p - 1), n**2 * c * p))
for k in range(1,p):
  for i in range(n):
    for j in range(c):
      index1 = k * n**2 * c + i * n * c + j * n
      Aw[(k - 1)*n*c + i*c + j] = np.block([
          np.zeros(index1),
          np.ones(n),
          np.zeros(n**2*c*p - index1 - n)
      ])

# Add zeros to not select the first phase w entries
Aw = np.block([
    [np.zeros((n*c, n**2*c*p))],
    [Aw]
    ])


# Select the y entries we want to match to the w entries
miniAy = np.diag(np.block([
    np.zeros(n*c),
    np.ones(n*c*(p-1))
    ]))

$(p-1) \times n \times c$ constraints.

Defining the variables (needed for $w$ constraints)

In [ ]:
x = cvx.Variable((N))

N2 = n*c*p
y = cvx.Variable((N2))

w = cvx.Variable((n*N2)) # w should in theory be of size n**2*c*(p-1) but we artificially extend it just to match dimensions with y


W constraints

In [ ]:
def vectorize_index(indexlist, vartype):
  if vartype =="x" : indexes_lengths = np.array([n**2,n,1])
  elif vartype=="y" : indexes_lengths = np.array([n*c,c,1])
  elif vartype=="w" : indexes_lengths = np.array([c*n**2,n*c,n,1])

  return np.array(indexlist) @ indexes_lengths

To represent the product between the entries of $x$ and $y$, the formula for $w$ is the following :
$$w_{k,i,j,e} \le x_{k,e,i}$$
$$w_{k,i,j,e} \le y_{k-1,e,j}$$
$$w_{k,i,j,e} \ge y_{k-1,e,j} + x_{k,e,i} -1$$

In [ ]:
wconstraints =[w>=0, w<=1]

for k in range(1,p):
  for i in range(n):
    for j in range(c):
      for e in range(n):
        wconstraints.extend([
          w[vectorize_index([k,i,j,e], "w")] <= x[vectorize_index([k,e,i], "x")],
          w[vectorize_index([k,i,j,e], "w")] <= y[vectorize_index([k-1,e,j], "y")],
          w[vectorize_index([k,i,j,e], "w")] >= y[vectorize_index([k-1,e,j], "y")] + x[vectorize_index([k,e,i], "x")] - 1
        ])

$(p-1) \times n \times c \times n \times  3$ constraints.

In [ ]:
yconstraints = [InitAy @ y == init_colors, # initial assignment
               FinalAy @ y == final_colors, # final assignment
               miniAy @ y == Aw @ w, # phase to phase correspondance for colors
                y>=0, y<=1]

### Solving

In [ ]:
cobj = np.ones((N))
ignorevalues = np.ravel([np.eye(n) for _ in range(p)])
cobj-=ignorevalues

objective = cvx.Minimize(cobj@x)


constraints = [A@x == b,
               x<=1, x>=0,
               allcols @ x <= 1]
constraints.extend(wconstraints)
constraints.extend(yconstraints)

prob = cvx.Problem(objective, constraints)

result= prob.solve()


print("Objective value :",prob.value)

xvalues = abs(
    np.reshape(
    np.round(x.value, 5),(p,n,n)))
print("\n X values : \n", xvalues)

yvalues = np.round(y.value, 5)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)


Objective value : 7.999999999883143

 X values : 
 [[[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]

 Y values : 

[[[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0

In [ ]:
def within_epsilon(x,y, epsilon=0.0001): return abs(x-y)<=epsilon

In [ ]:
def within_epsilon(x,y, epsilon=0.0001): return abs(x-y)<=epsilon

def plan_interpretation(xvalues, p, n):
  if type(xvalues) == cvx.expressions.variable.Variable :
    xvalues = np.array(xvalues.value)
  xvalues = xvalues.reshape((p,n,n))

  print("Number of moves :", round(xvalues.ravel() @ cobj),"\n")

  for t in range(1,p):
    print("Phase",t,":")
    for i in range(n):
      for j in range(n):
        if within_epsilon(xvalues[t,i,j],1) and j!=i:
          print("-Office",i+1,"moves to office",j+1)
    print("")

plan_interpretation(x,p,n)

Number of moves : 11 

Phase 1 :
-Office 2 moves to office 3
-Office 3 moves to office 1
-Office 4 moves to office 5
-Office 5 moves to office 4

Phase 2 :
-Office 1 moves to office 4
-Office 3 moves to office 1
-Office 4 moves to office 2

Phase 3 :
-Office 4 moves to office 3

Phase 4 :
-Office 5 moves to office 4

Phase 5 :
-Office 1 moves to office 2
-Office 2 moves to office 5



In [ ]:
def detail_phase(yvalues, phase):
  if type(yvalues) == cvx.expressions.variable.Variable :
    yvalues = np.array(yvalues.value)
  yvalues = yvalues.reshape((p,n,n))

  print("Phase",phase,"detailed :\n")

  for i in range(n):
    affected = 0
    for j in range(c):
      if within_epsilon(yvalues[phase,i,j], 1):
        print("Office",i+1,"belongs to", colors[j])
        affected=1
    if not affected : print("Office",i+1,"is unaffected")

detail_phase(y.value, 3)

Phase 3 detailed :

Office 1 belongs to MIDO
Office 2 belongs to LAMSADE
Office 3 belongs to the student association
Office 4 is unaffected
Office 5 belongs to the presidency


### Presidency constraints

To see if we can manage to solve the problem without the presidency and the student association neighboring each other. We are goint to represent each edge of the graph and make sure that, for every edge at every iteration, the two ends of that edge do not belong one to the presidency and the other to the student association.

In [ ]:
fiveOfficeGraph = [
    [0,1],
    [0,2],
    [1,3],
    [1,4],
    [2,3],
    [2,4]
]

nb_edges = len(fiveOfficeGraph)


PresidencyConstraints = np.empty((2*nb_edges*(p-1), N2))
for k in range(1,p-1):
  edgecount=0
  for edge in fiveOfficeGraph:
    cst1 = np.zeros(N2).reshape((p,n,c))
    cst1[k,edge[0], 3], cst1[k,edge[1], 4] = 1,1
    PresidencyConstraints[(k-1)*2*nb_edges + 2*edgecount] = cst1.ravel()
    edgecount+=1

    cst2 = np.zeros(N2).reshape((p,n,c))
    cst1[k,edge[1], 3], cst1[k,edge[0], 4] = 1,1
    PresidencyConstraints[(k-1)*2*nb_edges + 2*edgecount +1] = cst2.ravel()


In [ ]:
constraints.append(PresidencyConstraints @ y <= 1)
prob = cvx.Problem(objective, constraints)

result= prob.solve(solver  = cvx.OSQP)
print("Objective value :",prob.value)

print(prob.status)

if prob.status != "infeasible" :
  xvalues = abs(
      np.reshape(
      np.round(x.value, 1),(p,n,n)))
  print("\n X values : \n", xvalues)

  yvalues = np.round(y.value, 1)
  wvalues = np.round(w.value, 5)

  print("\n Y values : \n")

  yvalues = abs(
      np.reshape(yvalues,(p,n,c)) )
  print(yvalues)

Objective value : inf
infeasible


### Objective V2

In [ ]:
zf = np.block([final_colors for _ in range(4)])

zi = np.block([init_colors for _ in range(4)])

In [ ]:
cobj = np.ones((N))
ignorevalues = np.ravel([np.eye(n) for _ in range(p)])
cobj-=ignorevalues


lambbda = 100

objective2 = objective = cvx.Minimize(cobj@x
                         + lambbda * cvx.sum_squares(zi - y[n*c : 5*n*c]))

objective = cvx.Minimize(cobj@x)


constraints = [A@x == b,
               x<=1, x>=0,
               allcols @ x <= 1]
constraints.extend(wconstraints)
constraints.extend(yconstraints)

prob = cvx.Problem(objective2, constraints)


result= prob.solve(solver=cvx.ECOS)

print("Objective value :",prob.value)

xvalues = abs(
    np.reshape(
    np.round(x.value, 5),(p,n,n)))
print("\n X values : \n", xvalues)

yvalues = np.round(y.value, 5)
wvalues = np.round(w.value, 5)


print("\n Y values : \n")

yvalues = abs(
    np.reshape(yvalues,(p,n,c)) )
print(yvalues)


Objective value : 2007.999997535617

 X values : 
 [[[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]

 Y values : 

[[[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0